In [ ]:
!pip install -r requirements.txt
!pip install selenium webdriver_manager
! pip install selenium
!sudo apt-get update

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchAttributeException
from selenium.common.exceptions import ElementNotSelectableException
import time

driver = webdriver.Firefox()
try:
    driver.get("https://coinmarketcap.com/historical/20230825/")
except NoSuchDriverException as err:
    print("error occurred: {}".format(err))
time.sleep(1)
pos = 500
for i in range(0, 20):
    driver.execute_script("window.scrollTo(0, " + str(pos) + ")")
    pos += 500
    time.sleep(1)

In [2]:
rows = driver.find_elements(By.CSS_SELECTOR, ".cmc-table-row")
coins = []
for row in rows:
    try:
        coin = {}
        coin["rank"] = row.find_element(
            By.CSS_SELECTOR, ".cmc-table__cell--sort-by__rank div"
        ).text
        element = row.find_element(By.CSS_SELECTOR, ".cmc-table__column-name--name")
        coin["name"] = element.get_attribute("title")
        coin["symbol"] = row.find_element(
            By.CSS_SELECTOR, ".cmc-table__column-name--symbol"
        ).get_attribute("textContent")
        coin["circulating_supply"] = int(
            "".join(
                filter(
                    str.isdigit,
                    row.find_element(
                        By.CSS_SELECTOR,
                        ".cmc-table__cell--sort-by__circulating-supply div",
                    ).text,
                )
            )
        )
        coin["main_link"] = element.get_attribute("href")
        coin["historical_link"] = coin["main_link"] + "historical-data/"
        coins.append(coin)
    except NoSuchElementException as element_err:
        print("error occurred: {}".format(element_err))
    except NoSuchAttributeException as attribute_err:
        print("error occurred: {}".format(attribute_err))
    except ElementNotSelectableException as select_err:
        print("error occurred: {}".format(select_err))

driver.quit()

In [11]:

def get_coin_extras(driver, coin_url):
    try:
        driver.get(coin_url)
        try:
            github_link_element = driver.find_element(By.XPATH,"//a[contains(@href, 'github.com')]")
            github_link = github_link_element.get_attribute('href')
        except:
            github_link = ""
        try:
            driver.find_element(By.CSS_SELECTOR, ".sc-b7faf77f-1.kpTZfr").click()
            tags_elements = driver.find_elements(By.CSS_SELECTOR, ".cmc-modal .ctYAzo .cmc-link")
            tags = [tag_element.text for tag_element in tags_elements]
        except:
            tags_elements = driver.find_elements(By.CSS_SELECTOR, ".jaPKUl .ctYAzo .cmc-link")
            tags = [tag_element.text for tag_element in tags_elements]
    except NoSuchElementException:
        raise Exception("Failed to find elements")
    return {"github_link": github_link, "tags": tags}

In [4]:
import json
import pandas as pd

with open("coins_data.json", "w") as file:
    json.dump(coins, file)

df = pd.read_json("coins_data.json")
df

,rank,name,symbol,circulating_supply,main_link,historical_link
0,1,Bitcoin,BTC,19467775,https://coinmarketcap.com/currencies/bitcoin/,https://coinmarketcap.com/currencies/bitcoin/h...
1,2,Ethereum,ETH,120216444,https://coinmarketcap.com/currencies/ethereum/,https://coinmarketcap.com/currencies/ethereum/...
2,3,Tether USDt,USDT,82849133566,https://coinmarketcap.com/currencies/tether/,https://coinmarketcap.com/currencies/tether/hi...
3,4,BNB,BNB,153851122,https://coinmarketcap.com/currencies/bnb/,https://coinmarketcap.com/currencies/bnb/histo...
4,5,XRP,XRP,52914193551,https://coinmarketcap.com/currencies/xrp/,https://coinmarketcap.com/currencies/xrp/histo...
...,...,...,...,...,...,...
195,196,PlayDapp,PLA,560128667,https://coinmarketcap.com/currencies/playdapp/,https://coinmarketcap.com/currencies/playdapp/...
196,197,Pundi X (New),PUNDIX,258491637,https://coinmarketcap.com/currencies/pundix-new/,https://coinmarketcap.com/currencies/pundix-ne...
197,198,Vulcan Forged PYR,PYR,27182227,https://coinmarketcap.com/currencies/vulcan-fo...,https://coinmarketcap.com/currencies/vulcan-fo...
198,199,Status,SNT,3859423502,https://coinmarketcap.com/currencies/status/,https://coinmarketcap.com/currencies/status/hi...


In [ ]:
git = webdriver.Firefox()

extras_list = []

for ex in coins:
    extras = get_coin_extras(git,ex["main_link"])
    extras_list.append(extras)

In [ ]:
import pandas as pd

df1 = pd.DataFrame(extras_list)
#df1['name'] = [ex['name'] for ex in coins]
#df1['symbol'] = [ex['symbol'] for ex in coins]
df1['rank'] = [ex['rank'] for ex in coins]
new_cols = ["rank","github_link","tags"]
df1=df1[new_cols]
df1.to_csv('coins_extras.csv', index=False)